
# 1. No of columns in DS

In [ ]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error

In [ ]:
df = pd.read_parquet('/workspaces/mlopszoomcamp/1-intro/yellow_tripdata_2023-01.parquet')

In [ ]:
df.shape[1]

#  2. Computing duration and standard deviation


In [ ]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [ ]:
df['duration'].std()

# 3. Dropping outliers

In [ ]:
before_length = df.shape[0]
print (before_length)
df = df[(df.duration >= 1) & (df.duration <= 60)]
after_length = df.shape[0]
print (after_length)

In [ ]:
after_length / before_length * 100

# 4. One hot encoding

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
dv = DictVectorizer()
train_dicts = df[categorical].astype(str).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [ ]:
X_train.shape

#  5. Training The Model

In [ ]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


In [ ]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
ytrain_pred = lr.predict(X_train)
root_mean_squared_error(ytrain_pred, y_train)

#  6. Evaluating the model

In [ ]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)